In [1]:
%matplotlib qt
import matplotlib.pyplot as plt

import os.path as op
import numpy as np
import mne

mne.set_log_level('WARNING')

subjects_dir = op.expanduser("~/data/pic-name-data-bids/MRI/")
res_dir = op.expanduser("~/research/results/pic_name")

In [2]:
def average_stc(subjects, res_dir):
    stc_fnames = []
    for subject in subjects:
        stc_fnames.append(op.join(res_dir, subject + '/dspm_%s' % subject))
    stcs = [mne.read_source_estimate(stc_fname) for stc_fname in stc_fnames]
    return np.mean(stcs)

def get_label(subjects_dir):
    labels = mne.read_labels_from_annot('fsaverage', 'HCPMMP1', 'both',
                                        subjects_dir=subjects_dir)
    labels_vis = []
    ROI = ['L_V1_ROI-lh', 'R_V1_ROI-rh']
    for lbl in labels:
        if lbl.name in ROI:
            labels_vis.append(lbl)
    label = labels_vis[0]
    for i in range(1, len(labels_vis)):
        label = label + labels_vis[i]
    return label

In [3]:
bili = [2, 8, 9, 12]
clil = [6, 10, 11, 15]
mono = [3, 4, 5, 13, 14]
subjects_bili = ["sub-%02d" % i for i in bili]
subjects_clil = ["sub-%02d" % i for i in clil]
subjects_mono = ["sub-%02d" % i for i in mono]

In [4]:
label = get_label(subjects_dir)
average_stc_bili = average_stc(subjects_bili, res_dir)
average_stc_clil = average_stc(subjects_clil, res_dir)
average_stc_mono = average_stc(subjects_mono, res_dir)

In [5]:
average_stc_bili_label = average_stc_bili.in_label(label)
bili_mean = np.mean(average_stc_bili_label.data, axis=0)
average_stc_clil_label = average_stc_clil.in_label(label)
clil_mean = np.mean(average_stc_clil_label.data, axis=0)
average_stc_mono_label = average_stc_mono.in_label(label)
mono_mean = np.mean(average_stc_mono_label.data, axis=0)

In [6]:
times = 1e3 * average_stc_bili_label.times # times in ms
plt.figure()
h0 = plt.plot(times, mono_mean, 'b', linewidth=3)
h1, = plt.plot(times, bili_mean, 'g', linewidth=3)
h1, = plt.plot(times, clil_mean, 'r', linewidth=3)
plt.show()

In [ ]:
initial_time = 0.24
surfer_kwargs = dict(subject="fsaverage",
                     background="white", foreground='black',
                     cortex=("gray", -1, 6, True), smoothing_steps=10,
                     clim=dict(kind='value', lims=[2, 5, 6]),
                     time_viewer=False)

In [ ]:
brain_bili = average_stc_bili.plot(initial_time=initial_time, subjects_dir=subjects_dir, 
                              hemi="both", views='dor', **surfer_kwargs)

In [ ]:
brain_mono = average_stc_mono.plot(initial_time=initial_time, subjects_dir=subjects_dir, 
                              hemi="both", views='dor', **surfer_kwargs)

In [ ]:
brain_clil = average_stc_clil.plot(initial_time=initial_time, subjects_dir=subjects_dir, 
                              hemi="both", views='dor', **surfer_kwargs)

In [ ]:
# brain.save_image(op.join(res_dir, "dspm_fsavg_t=400ms.png"))